In [1]:
import os
from glob import glob
from skimage.metrics import peak_signal_noise_ratio
import torch
import numpy as np
from PIL import Image

from notebook_helpers import get_model, run, get_cond_for_psnr_testing

c:\Users\oKata\anaconda3\envs\ldm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATASET_PATH = None
SAVE_PATH = None
DIFFUSION_STEPS = 100
model = get_model('superresolution')

Using downloaded and verified file: logs\diffusion\superresolution_bsr\configs\project.yaml\dl=1
Using downloaded and verified file: logs\diffusion\superresolution_bsr\checkpoints\last.ckpt\dl=1
Loading model from logs\diffusion\superresolution_bsr\checkpoints\last.ckpt\dl=1
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 113.62 M params.
Keeping EMAs of 308.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [3]:
image_filenames = glob(os.path.join(DATASET_PATH, '*'))

In [4]:
image_filenames = image_filenames[0:1]

In [7]:
def extract_sample(logs) -> Image:
    sample = logs["sample"]
    sample = sample.detach().cpu()
    sample = torch.clamp(sample, -1., 1.)
    sample = (sample + 1.) / 2. * 255
    sample = sample.numpy().astype(np.uint8)
    sample = np.transpose(sample, (0, 2, 3, 1))
    return sample[0]


def extract_orig(example):
    orig_image = np.asarray(example['orig'])[0]
    return (orig_image * 255).astype(np.uint8)


def estimate_psnr(image_filenames, model, diff_steps, save_path=None):
    psnr_vals = []
    for image_file in image_filenames:
        example = get_cond_for_psnr_testing('superresolution', image_file)
        logs = run(model, example, 'superresolution', diff_steps)
        
        orig_image = extract_orig(example)
        test_image = extract_sample(logs)
        psnr = peak_signal_noise_ratio(orig_image, test_image)
        psnr_vals.append(psnr)
        
        if save_path is not None:
            image_name = image_file.split(os.sep)[-1]
            savepath = os.path.join(save_path, image_name)
            test_image.save(save_path)
    return np.mean(psnr_vals)

In [8]:
psnr = estimate_psnr(image_filenames, model, DIFFUSION_STEPS, SAVE_PATH)

Plotting: Switched to EMA weights
Sampling with eta = 1.0; steps: 100
Data shape for DDIM sampling is (1, 3, 339, 510), eta 1.0
Running DDIM Sampling with 100 timesteps


DDIM Sampler: 100%|██████████| 100/100 [05:11<00:00,  3.12s/it]


Plotting: Restored training weights
